# Created an interactive map for CRESST/GAIA webpage 

Goals:
- do not embed data (use APIs to load any data on-demand)
- keep it simple :) 

In [ ]:
import folium
import geopandas as gpd
import urllib.parse
import branca
from folium.elements import MacroElement
from jinja2 import Template
import xyzservices
import numpy as np
import matplotlib
import matplotlib.colors as mcolors
import branca.colormap as bcm

In [ ]:
# Restrict map to WA State
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
minlon, minlat, maxlon, maxlat = aoi.total_bounds

In [ ]:
m = folium.Map(
    min_lat= minlat - 1,
    max_lat= maxlat + 1,
    min_lon= minlon - 1,
    max_lon= maxlon + 1,
    max_bounds=True,
    # # NOTE: something seems to be override this...
    min_zoom=7,
    max_zoom=12,
    zoom_start=7,
    control_scale = True,
)

## Stations

In [ ]:

# Keep track of layer name and layer in a dictionary
station_layers = {}

# Add In-Situ Stations
stations = {'Precipitation': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/precip-stations-wa-styled.geojson",
            'Streamflow': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/streamflow-stations-wa-styled.geojson",
            'Seismometer': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/seismic-stations-wa-styled.geojson"}

for name, url in stations.items():

    # Get list of columns for popup
    gf = gpd.read_file(url)
    drop_cols = ['marker-size', 'marker-symbol', 'marker-color', 'geometry']
    all_fields = list(gf.drop(columns=drop_cols).columns)
    color = gf['marker-color'].iloc[0]
    hover_label = 'stid' if name != 'Seismometer' else 'station'

    mymarker = folium.Circle(fill=False, radius=200, color=color)
    gjson = folium.GeoJson(url,
                           embed=False,
                           marker=mymarker,
                           name=f"{name}").add_to(m)

    station_layers[name] = gjson

    folium.features.GeoJsonTooltip(fields=[hover_label], labels=False).add_to(gjson)
    folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

# Basemaps

To see what is out there: https://xyzservices.readthedocs.io/en/stable/

In [ ]:
# Simplify adding tiles w/ XYZ services
xyzservices.providers.Esri#.WorldImagery

In [ ]:
# Keep track of layer name and layer in a dictionayr
basemap_layers = {}

# Default Basemap = CartoDB Tiles
# =============
tiles = folium.TileLayer(
    tiles=xyzservices.providers.CartoDB.Positron,
    overlay=False,
    show=True,
    control=True
).add_to(m)

basemap_layers['CartoDB'] = tiles

# ESRI Tiles
# =============
#ESRI_API="https://server.arcgisonline.com/ArcGIS/rest/services/{variant}/MapServer/tile/{z}/{y}/{x}"
for variant in ['WorldImagery','WorldShadedRelief','WorldTopoMap','WorldGrayCanvas']:
    layer_name = f"ESRI {variant}"
    tiles = folium.TileLayer(
        tiles=xyzservices.providers.Esri[variant],
        name=layer_name,
        overlay=False,
        show=False,
        control=True
    ).add_to(m)

    basemap_layers[layer_name] = tiles

# NASA GIBS
# =============
# A lot of available layers, just using DEM for starters
# https://worldview.earthdata.nasa.gov
# https://www.earthdata.nasa.gov/engage/open-data-services-software/earthdata-developer-portal/gibs-api
layer_name = "ASTER GDEM Greyscale Shaded Relief"
tiles = folium.raster_layers.WmsTileLayer(
    url="https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi",
    layers="ASTER_GDEM_Greyscale_Shaded_Relief",
    name=layer_name,
    fmt="image/png",
    transparent=True,
    overlay=False,
    control=True,
    show=False,
).add_to(m)

basemap_layers[layer_name] = tiles

# Example for mult-year prodcts
#years = [2023,2024,2025]
years = [2025]
for year in years:
    layer_name = f"OPERA Annual Disturbance {year}"
    tiles = folium.raster_layers.WmsTileLayer(
        #url="https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi",
        url=f"https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?TIME={year}-01-01",
        layers="OPERA_L3_DIST-ANN-HLS_Color_Index",
        name=layer_name,
        fmt="image/png",
        #transparent=True,
        overlay=False,
        control=True,
        show=False,
        # NOT working.... need to pass in URL param directly
        #extra_wms_params={
        #    "TIME": "2023-01-01"
            #"TIME": "2024-06-01T00:00:00Z"
        #},
    ).add_to(m)

    basemap_layers[layer_name] = tiles

# TODO: legends for sophisticated layers
# https://gibs.earthdata.nasa.gov/wms/epsg3857/best/?request=GetMetadata&layer=OPERA_L3_DIST-ANN-HLS_Color_Index
# Apparently has a legend URL! https://gibs.earthdata.nasa.gov/legends/OPERA_Vegetation_Disturbance_Annual_H.png
# legend_url = "https://gibs.earthdata.nasa.gov/legends/OPERA_Vegetation_Disturbance_Annual_H.png"
# legend_html = f"""
# <div style="
#     position: fixed;
#     bottom: 50px; left: 50px; width: 300px; z-index:9999;
#     background-color: white; padding: 10px; border:2px solid grey;
#     box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
#     <b>OPERA Annual Disturbance 2024 Legend</b><br>
#     <img src="{legend_url}" style="width:100%;"/>
# </div>
# """
# m.get_root().html.add_child(Element(legend_html))

# WA DNR 

In [ ]:
# url = "https://gis.dnr.wa.gov/site3/rest/services/Public_Wildfire/WADNR_PUBLIC_WD_WildFire_Data/MapServer/0/query"
# params = {
#     #'where': 'YEAR = 2024',
#     'where': 'YEAR >= 2023',
#     #'where': 'YEAR >= 2021',   #> maxRecordCount
#     'outFields': '*',
#     'f': 'geojson'
# }
# query_string = urllib.parse.urlencode(params)
# full_url = f"{url}?{query_string}"
# print(full_url)
# #fires = gpd.read_file(full_url)

# gjson = folium.GeoJson(full_url,
#                         embed=False,
#                         show=False,
#                         name="WA DNR Wildfire boundaries (>=2023)",
#                         style_function=lambda x: {
#                             'fillColor': 'none',
#                             'color': 'red',
#                             'weight': 2,
#                             'fillOpacity': 0
#                         }).add_to(m)

# all_attributes = ['OBJECTID', 'UNITID', 'AGENCY', 'FIRENAME', 'FIRENUM', 'STARTDATE', 'PERIMDATE', 'ACRES', 'YEAR', 'CAUSE', 'SHAPE.AREA', 'SHAPE.LEN', 'geometry']
# show_attributes = ['FIRENAME', 'YEAR', 'STARTDATE', 'PERIMDATE', 'SHAPE.AREA',  'CAUSE']
# folium.features.GeoJsonTooltip(fields=["FIRENAME"], labels=False).add_to(gjson)
# folium.features.GeoJsonPopup(fields=show_attributes, labels=True).add_to(gjson);

## Contextual GIS layers 

In [ ]:
gis_layers = {}
# Add Google Roads layer (optional)
tiles = folium.raster_layers.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=h&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Roads',
    overlay=True,
    control=True,
    show=False,
).add_to(m);

gis_layers['Google Roads'] = tiles

In [ ]:
# Watersheds
# Get GeoJSON from an API
huc8 = 'https://hydro.nationalmap.gov/arcgis/rest/services/wbd/FeatureServer/4/query?where=1%3D1&geometry=-124%2C45%2C-116%2C49&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outFields=name%2Chuc8%2Careasqkm&outSR=4326&f=geojson&maxAllowableOffset=0.01'

gjson = folium.GeoJson(huc8,
                        embed=False,
                        show=False,
                        name="Watershed Sub-basins (HUC8)",
                        style_function=lambda x: {
                            'fillColor': 'blue', # 'none' = no fill
                            'color': 'blue',
                            'weight': 2,
                            'fillOpacity': 0.2, #0 = totally transpatent
                        }).add_to(m)

gis_layers['Watershed Sub-basins (Huc8)'] = gjson

folium.features.GeoJsonTooltip(fields=["name"], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=["name", "huc8", "areasqkm"], labels=True).add_to(gjson);


## Dynamic tiling of tiffs


for these we need a colorbar to interpret the rendered tiff...

In [ ]:
# Colorbar for titiler raster overlays
class ToggleableColorbar(MacroElement):
    def __init__(self, layer_name, colormap):
        super().__init__()
        self.layer_name = layer_name
        self.colormap = colormap

        self._template = Template("""
        {% macro script(this, kwargs) %}
        var legend = L.control({position: 'bottomright'});
        legend.onAdd = function (map) {
            var div = L.DomUtil.create('div', 'info legend');
            div.style.backgroundColor = 'white';
            div.style.padding = '10px';
            div.innerHTML = `{{ this.colormap._repr_html_() }}`;
            return div;
        };
        legend.addTo({{ this._parent.get_name() }});
        legend.getContainer().style.display = 'none';

        {{ this._parent.get_name() }}.on('overlayadd', function(e) {
            if (e.name === '{{ this.layer_name }}') {
                legend.getContainer().style.display = 'block';
            }
        });

        {{ this._parent.get_name() }}.on('overlayremove', function(e) {
            if (e.name === '{{ this.layer_name }}') {
                legend.getContainer().style.display = 'none';
            }
        });
        {% endmacro %}
        """)

### SOLUS

In [ ]:
# Keep track of all COG layers for Dynamic Tiling
cog_layers = {}

tif = "https://storage.googleapis.com/solus100pub/anylithicdpt_cm_p.tif"
tiler = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
layer_name = 'SOLUS100 Soil Depth'

virtual_tiles = f"{tiler}?url={tif}"
tiles = folium.TileLayer(tiles=virtual_tiles,
                 overlay=True,
                 control=True,
                 show=False,
                 name=layer_name,
                 attr="USDA").add_to(m)

cog_layers[layer_name] = tiles

cm = branca.colormap.linear.Greys_06.scale(0,1)
cm.colors.reverse()
cm = branca.colormap.LinearColormap(colors=cm.colors, caption='SOLUS100 soil depth (cm)', vmin=0, vmax=200)

# Link colorbar visibility to layer visibility
m.add_child(ToggleableColorbar(layer_name, cm));

### YiYU Seismic Velocities

In [ ]:
def mpl_to_branca(cmap, vmin=0, vmax=1, n=256):
    """
    Convert a matplotlib colormap to a branca LinearColormap.

    Parameters
    ----------
    cmap : matplotlib colormap or str
        Colormap instance or name (e.g. 'viridis')
    vmin, vmax : float
        Data range for the branca colormap
    n : int
        Number of color samples

    Returns
    -------
    branca.colormap.LinearColormap
    """
    if isinstance(cmap, str):
        cmap = matplotlib.colormaps.get_cmap(cmap)

    colors = [
        mcolors.to_hex(cmap(i))
        for i in np.linspace(0, 1, n)
    ]

    return bcm.LinearColormap(
        colors=colors,
        vmin=vmin,
        vmax=vmax
    )

In [ ]:
# Test visualization
# Get a sense of colorscale

# import rioxarray as rxr
# import matplotlib.pyplot as plt
# da =rxr.open_rasterio('global_vs30_2.tif', overview_level=1).squeeze()
# da = da.sel(x=slice(minlon, maxlon), y=slice(maxlat, minlat))
# #da.plot.imshow(cmap='plasma_r') # defaul scaling ~100 to 900
# #da.plot.imshow(cmap='plasma_r', vmin=200, vmax=800) # ~300-700
# #da.plot.imshow(cmap='plasma_r', robust=True)
# plt.title('Vs30');

In [ ]:
# Set up scaling and tile rendering
layer_name = "Vs30"
attribution = "USGS"
colormap = "plasma_r"
vmin = 100
vmax = 800
branca_cmap = mpl_to_branca(colormap, vmin=vmin, vmax=vmax)
branca_cmap.caption = "Vs30 Global (m/s)"

tif = "https://dasway.ess.washington.edu/shared/niyiyu/global_vs30_2.tif"
tiler = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles = f"{tiler}?url={tif}&rescale={vmin},{vmax}&colormap_name={colormap}"

# Display the colormap
branca_cmap

In [ ]:
tiles_vs30 = folium.TileLayer(tiles=virtual_tiles,
                 overlay=True,
                 control=True,
                 show=False,
                 name=layer_name,
                 attr=attribution).add_to(m)

cog_layers[layer_name] = tiles_vs30

# Add the colormap to the map as a control
#cm.add_to(m)
# Link colorbar visibility to layer visibility
m.add_child(ToggleableColorbar(layer_name, branca_cmap));

# Optional Plugins

- Ability to measure distances 
- Ability to draw vector shapes to export (e.g. study area AOI)

In [ ]:
# Optional plugins
from folium.plugins import Draw
from folium.plugins import MeasureControl

# Position options position : {'topleft', 'toprigth', 'bottomleft', 'bottomright'}
# https://github.com/python-visualization/folium/issues/1806
Draw(export=True, position='bottomleft').add_to(m)

m.add_child(MeasureControl(position='topleft'));

In [ ]:
from folium.plugins.treelayercontrol import TreeLayerControl

In [ ]:
#TreeLayerControl?

In [ ]:
basemap_tree = {
        "label": "Basemaps",
        "select_all_checkbox": False,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in basemap_layers.items()]
        }

overlay_tree = {
    "label": "All Layers",
    "select_all_checkbox": "Un/select all",
    "children": [
        {
        "label": "In Situ Sensors",
        "select_all_checkbox": True,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in station_layers.items()]
        },
        # TODO: turn into radioGroup so that only one can be selected at a time?
        {
        "label": "Raster Layers",
        "select_all_checkbox": False,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in cog_layers.items()]
        },
        {
        "label": "GIS Layers",
        "select_all_checkbox": False,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in gis_layers.items()]
        }
    ]
    }

TreeLayerControl(base_tree=basemap_tree, overlay_tree=overlay_tree).add_to(m);

## Display and save map

In [ ]:
# Save the map to an HTML file
# Todo: use from folium.plugins.treelayercontrol import TreeLayerControl
# https://python-visualization.github.io/folium/latest/user_guide/plugins/treelayercontrol.html
#folium.LayerControl(collapsed=True, draggable=True).add_to(m)

m

In [ ]:
m.save('index.html')